In [58]:
import pandas as pd
import ast
import json


In [4]:
paid = pd.read_csv('Данные/paid_all_period.csv')
affected = pd.read_csv('Данные/sectors_cluster_affected_total_all_period.csv')
targeted = pd.read_csv('Данные/sectors_cluster_targeted_total_all_period.csv')

In [6]:
paid.head()

,Unnamed: 0,Year,Donor Name,CBPF Name,Paid Amount,Pledged Amount,Total Contributions,Local Curency,Exchange Rate,Paid Amount (Local Currency),Pledged Amount (Local Currency),Total Contributions (Local Currency)
0,0,2006,Netherlands,Ethiopia,5012000.0,0.0,5012000.0,USD,1.00,5012000.00,0,5012000.00
1,1,2006,Norway,Ethiopia,1553277.0,0.0,1553277.0,USD,1.00,1553277.00,0,1553277.00
2,2,2006,Switzerland,Ethiopia,247934.0,0.0,247934.0,CHF,1.21,300000.14,0,300000.14
3,3,2006,United Kingdom,Ethiopia,8861284.0,0.0,8861284.0,GBP,0.56,5000000.20,0,5000000.20
4,0,2007,Netherlands,Ethiopia,3395940.0,0.0,3395940.0,USD,1.00,3395940.00,0,3395940.00


In [17]:
paid_money_by_country = pd.DataFrame(paid['CBPF Name'].value_counts())
paid_money_by_country['total_contributions'] = paid.groupby(by=['CBPF Name']).sum()['Total Contributions']
paid_money_by_country.sort_values(by='total_contributions')


,CBPF Name,total_contributions
Niger (RhPF),6,8.764908e+06
Burkina Faso (RhPF),6,1.228212e+07
Colombia,24,1.270318e+07
Venezuela,11,1.276383e+07
Jordan,35,6.183625e+07
Myanmar,65,9.210216e+07
Haiti,47,9.386667e+07
Pakistan,33,1.033629e+08
Lebanon,60,1.105090e+08
Nigeria,61,1.512291e+08


In [18]:
paid_money_by_donor = pd.DataFrame(paid['Donor Name'].value_counts())
paid_money_by_donor['total_contributions'] = paid.groupby(by=['Donor Name']).sum()['Total Contributions']
paid_money_by_donor.sort_values(by='total_contributions')

,Donor Name,total_contributions
Kenya,1,1.987330e+03
Madagascar,1,5.000000e+03
Sri Lanka,4,5.000000e+03
Chile,1,5.000000e+03
Montenegro,1,1.234568e+04
...,...,...
Norway,115,4.384337e+08
Netherlands,97,7.962476e+08
Sweden,177,9.000405e+08
Germany,111,1.073204e+09


In [113]:
def count_degree_paid(data, 
                      year_begin=2020, 
                      year_finish=2021, 
                      count_type='unique_country'):
    

    data_period = data[(data.Year >= year_begin) & (data.Year <= year_finish)]
    
    if count_type == 'unique_country':
        data_country = data_period[['Donor Name', 'CBPF Name']].drop_duplicates()
        donor_weight = pd.DataFrame(data_country.groupby(by=['Donor Name']).count()['CBPF Name'])
        donor_weight['transaction_sum'] = data_period.groupby(by='Donor Name').sum()['Total Contributions']
        donor_weight['transaction_number'] = data_period.groupby(by='Donor Name').count()['Total Contributions']
        donor_weight = donor_weight.reset_index()
        donor_weight['help_countries'] = ''

        poor_weight = pd.DataFrame(data_country.groupby(by=['CBPF Name']).count()['Donor Name'])
        poor_weight['transaction_sum'] = data_period.groupby(by='CBPF Name').sum()['Total Contributions']
        poor_weight['transaction_number'] = data_period.groupby(by='CBPF Name').count()['Total Contributions']
        poor_weight = poor_weight.reset_index()
        poor_weight['help_countries'] = ''

        for donor in data_country['Donor Name'].unique():
            help_countries = data_country[data_country['Donor Name'] == donor]['CBPF Name'].values
            help_countries = ",".join(help_countries)
            donor_weight.loc[donor_weight['Donor Name'] == donor, 'help_countries'] = help_countries

        for poor in data_country['CBPF Name'].unique():
            help_countries = data_country[data_country['CBPF Name'] == poor]['Donor Name'].values
            help_countries = ",".join(help_countries)
            poor_weight.loc[poor_weight['CBPF Name'] == poor, 'help_countries'] = help_countries
        
        donor_weight['help_countries'] = donor_weight['help_countries'].apply(lambda x: x.split(','))
        poor_weight['help_countries'] = poor_weight['help_countries'].apply(lambda x: x.split(','))

        donor_weight = donor_weight.rename(columns={'CBPF Name': 'cbpf_number'})
        poor_weight = poor_weight.rename(columns={'Donor Name': 'donor_number'})

        return donor_weight, poor_weight



In [114]:
donor_weight, poor_weight = count_degree_paid(paid, year_begin=2016, year_finish=2021)


In [115]:
poor_weight

,CBPF Name,donor_number,transaction_sum,transaction_number,help_countries
0,Afghanistan,16,3.004483e+08,48,"[Australia, Denmark, Germany, Netherlands, Nor..."
1,CAR,15,1.433469e+08,54,"[Belgium, Canada, Denmark, Germany, Ireland, L..."
2,Colombia,4,2.706420e+06,7,"[Spain, Sweden, Switzerland, PRIVATE SECTOR]"
3,DRC,11,3.391533e+08,45,"[Belgium, Ireland, Luxembourg, Netherlands, Sw..."
4,Ethiopia,17,3.427118e+08,52,"[Australia, Denmark, Germany, Ireland, Korea, ..."
5,Iraq,19,2.923819e+08,55,"[Australia, Belgium, Canada, Cyprus, Denmark, ..."
6,Jordan,11,4.880278e+07,28,"[Belgium, Ireland, Netherlands, Sweden, United..."
7,Lebanon,15,7.127652e+07,42,"[Belgium, Germany, Ireland, Netherlands, Swede..."
8,Myanmar,15,6.360746e+07,43,"[Australia, Denmark, Sweden, Switzerland, Unit..."
9,Nigeria,21,1.313785e+08,52,"[Azerbaijan, Belgium, Canada, Denmark, Germany..."
